# From normalize notebook

In [5]:
import geopandas as gpd
import time
import zipfile
from getpass import getpass
from glob import glob

import pygbif.occurrences as occ
import pygbif.species as species
import requests

import earthpy
import os
import pathlib
import pandas as pd


In [4]:
%store -r gbif_df eco_gdf

The code below was given by Lilly to make the data set smaller and more managable 
Had to change gbif_gdf to gbif_df
Havent changed the number yet but it was able to run.

In [3]:
# Fast bounding box filter
na_gbif_gdf = gbif_df[
    (gbif_df.decimalLongitude >= -170) &
    (gbif_df.decimalLongitude <= -50) &
    (gbif_df.decimalLatitude >= 5) &
    (gbif_df.decimalLatitude <= 85)
].copy()

NameError: name 'gbif_df' is not defined

In [6]:
gbif_gdf = gpd.GeoDataFrame(gbif_df,
    geometry=gpd.points_from_xy(gbif_df.decimalLongitude, gbif_df.decimalLatitude),
    crs="EPSG:4326"
)
gbif_df
print(gbif_gdf.head)

<bound method NDFrame.head of             decimalLatitude  decimalLongitude  month  \
gbifID                                                 
4923516522        41.852812        -87.611721      5   
4444336266        29.755600        -95.739600      4   
4444338185        29.950300        -95.445200      3   
4444335183        29.629800        -95.323900      4   
5196047463        40.003844       -105.028095      5   
...                     ...               ...    ...   
4173223348        44.366800        -83.563700      5   
4444338186        29.806200        -95.557400      4   
4444336188        29.955200        -95.560000     10   
4444336173        30.131200        -95.778500      2   
4444335096        30.146574        -95.642283      4   

                              geometry  
gbifID                                  
4923516522  POINT (-87.61172 41.85281)  
4444336266    POINT (-95.7396 29.7556)  
4444338185    POINT (-95.4452 29.9503)  
4444335183    POINT (-95.3239 29.629

In [7]:
%store -r gbif_gdf
gbif_gdf.columns
gbif_gdf['geometry']


gbifID
4158712344      POINT (-82.5092 41.9137)
4923515059    POINT (-87.61172 41.85281)
4923522410    POINT (-87.61172 41.85281)
4923520798    POINT (-87.61172 41.85281)
4923520314    POINT (-87.63013 41.88036)
                         ...            
4423534780      POINT (-75.1254 40.0626)
4524632357      POINT (-89.8049 44.6967)
4173211734      POINT (-82.4753 42.0478)
4173216429      POINT (-74.5468 46.1638)
4423531794      POINT (-75.1254 40.0626)
Name: geometry, Length: 165614, dtype: geometry

In [8]:
gbif_ecoregion_gdf = gpd.sjoin(
    gbif_gdf.to_crs(epsg=4326),
    eco_gdf.to_crs(epsg=4326),
    how='inner',
    predicate='intersects'
)

In [11]:
eco_gdf

,name,area,geometry
ecoregion_id,,,
1.0,Adelie Land tundra,0.038948,"MULTIPOLYGON (((158.7141 -69.60657, 158.71264 ..."
2.0,Admiralty Islands lowland rain forests,0.170599,"MULTIPOLYGON (((147.28819 -2.57589, 147.2715 -..."
3.0,Aegean and Western Turkey sclerophyllous and m...,13.844952,"MULTIPOLYGON (((26.88659 35.32161, 26.88297 35..."
4.0,Afghan Mountains semi-desert,1.355536,"MULTIPOLYGON (((65.48655 34.71401, 65.52872 34..."
5.0,Ahklun and Kilbuck Upland Tundra,8.196573,"MULTIPOLYGON (((-160.26404 58.64097, -160.2673..."
...,...,...,...
848.0,Sulawesi lowland rain forests,9.422097,"MULTIPOLYGON (((117.33111 -7.53306, 117.30525 ..."
212.0,East African montane forests,5.010930,"MULTIPOLYGON (((36.7375 -3.13, 36.7375 -3.1316..."
224.0,Eastern Arc forests,0.890325,"MULTIPOLYGON (((36.38 -8.96583, 36.38 -8.96667..."


In [10]:
gbif_ecoregion_gdf

,month,geometry,ecoregion_id,name,area
gbifID,,,,,
4923515059,5,POINT (-87.61172 41.85281),140.0,Central US forest-grasslands transition,24.173692
4923522410,9,POINT (-87.61172 41.85281),140.0,Central US forest-grasslands transition,24.173692
4923520798,5,POINT (-87.61172 41.85281),140.0,Central US forest-grasslands transition,24.173692
4923520314,9,POINT (-87.63013 41.88036),140.0,Central US forest-grasslands transition,24.173692
4923522547,8,POINT (-87.61172 41.85281),140.0,Central US forest-grasslands transition,24.173692
...,...,...,...,...,...
4135517885,5,POINT (-83.8394 41.5434),679.0,Southern Great Lakes forests,24.575679
4423534780,9,POINT (-75.1254 40.0626),504.0,Northeast US Coastal forests,7.876180
4524632357,6,POINT (-89.8049 44.6967),764.0,Upper Midwest US forest-savanna transition,15.481685


In [15]:
# Identify ecoregion for each observation
gbif_ecoregion_gdf = (
    eco_gdf
    .to_crs(gbif_gdf.crs)
    .sjoin(
        gbif_gdf,
        how='inner',
        predicate='contains')
    # Select the required columns
    [['month', 'name']]
    )
gbif_ecoregion_gdf

,month,name
ecoregion_id,,
13.0,5,Alberta-British Columbia foothills forests
13.0,5,Alberta-British Columbia foothills forests
13.0,6,Alberta-British Columbia foothills forests
13.0,7,Alberta-British Columbia foothills forests
13.0,6,Alberta-British Columbia foothills forests
...,...,...
845.0,9,North Atlantic moist mixed forests
845.0,10,North Atlantic moist mixed forests
845.0,9,North Atlantic moist mixed forests


Replaced code below with the one above, something to do with the code calling the incorect variable. 
Leave it be for now, and move on.

In [47]:
gbif_ecoregion_gdf = (
    gbif_gdf.to_crs(epsg=4326)
    # Match the CRS of the GBIF data and the ecoregions
    
    # Find ecoregion for each observation
    .sjoin( 
        gbif_gdf,
        eco_gdf,
        how='inner', 
        predicate='intersects')
)
gbif_ecoregion_gdf

TypeError: GeoDataFrame.sjoin() got multiple values for argument 'how'

In [9]:
gbif_ecoregion_gdf.columns
eco_gdf.columns

Index(['name', 'area', 'geometry'], dtype='object')

In [50]:
gbifID_columns = gbif_ecoregion_gdf.columns[gbif_ecoregion_gdf.columns.str.startswith('gbifID')]

In [13]:
gbif_gdf

,month,geometry
gbifID,,
4158712344,5,POINT (-82.5092 41.9137)
4923515059,5,POINT (-87.61172 41.85281)
4923522410,9,POINT (-87.61172 41.85281)
4923520798,5,POINT (-87.61172 41.85281)
4923520314,9,POINT (-87.63013 41.88036)
...,...,...
4423534780,9,POINT (-75.1254 40.0626)
4524632357,6,POINT (-89.8049 44.6967)
4173211734,5,POINT (-82.4753 42.0478)


In [51]:
occurrence_gdf = (
    gbif_ecoregion_gdf
    # Select only necessary columns
    # Reset index
    #.reset_index()
    # For each ecoregion, for each month...
    .groupby(['gbifID', 'month'])
    # ...count the name of occurrences
    .agg
    (occurrences=('gbifID', 'count')),
    # area= ('area' ,'first'))
)
 
# Normalize by area
#occurrence_df['density'] = (
 #   occurrence_df.occurrences
  #  / occurrence_df.area

# Get rid of rare observations (possible misidentification?)
occurrence_gdf = occurrence_gdf[occurrence_gdf]>1

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
    occurrence_gdf
    .groupby('ecoregion_id')
    .mean()
)
# Take the mean by month
mean_occurrences_by_month = (
    occurrence_gdf
    .groupby('month')
    .mean
    
)

KeyError: "Column(s) ['gbifID'] do not exist"

In [ ]:
# Normalize by space and time for sampling effort
# Was stuck on this becuase a lack of famliarity 
# Got past it and will remember for net time 
occurrence_df['norm_occurrences'] = (
    occurrence_df[['density']]
    /mean_occurrences_by_ecoregion[['density']]
    /mean_occurrences_by_month[['density']]
)
occurrence_df

NameError: name 'occurrence_df' is not defined

In [ ]:
occurrence_df.reset_index().plot.scatter(
    x='month' , y='norm_occurrences' , c='ecoregion_id',
    logy=True
)

NameError: name 'occurrence_df' is not defined

# From plot note book

In [ ]:
# Simplify the geometry to speed up processing
eco_gdf.geometry = eco_gdf.simplify(.01, preserve_topology=False)
# Change the CRS to Mercator for mapping
eco_gdf = eco_gdf.to_crs(ccrs.Mercator())

# Check that the plot runs in a reasonable amount of time
eco_gdf.hvplot(geo=True, crs=ccrs.Mercator())

In [ ]:
occurrence_df[['norm_occurrences']].head()

In [ ]:
# Join the occurrences with the plotting GeoDataFrame
occurrence_gdf = eco_gdf.join(occurrence_df[['norm_occurrences']])

# Get the plot bounds so they don't change with the slider
xmin, ymin, xmax, ymax = gbif_gdf.to_crs(ccrs.Mercator()).total_bounds

# Plot occurrence by ecoregion and month
migration_plot = (
    occurrence_gdf
    .to_crs(ccrs.Mercator())
    .hvplot(
        c='norm_occurrences',
        groupby='month',
        # Use background tiles
        geo=True, crs=ccrs.Mercator(), tiles='CartoLight',
        title="Veery Migration",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=600,
        widget_location='bottom'
    )
)

#Save the plot
migration_plot.save('migration.html', embed=True)

# Show the plot
migration_plot

NameError: name 'eco_gdf' is not defined